<a href="https://colab.research.google.com/github/thunguyen177/DPER/blob/main/DPER_multiple_class_without_equal_covariance_assumption__FASHION_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## libraries and function 

In [ ]:
!pip install impyute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
import impyute as impy
from fancyimpute import IterativeSVD, SoftImpute, NuclearNormMinimization
import pandas as pd
import time 
from missingpy import MissForest

distutils: /home/thu/.conda/envs/tf2/include/python3.8/UNKNOWN
sysconfig: /home/thu/.conda/envs/tf2/include/python3.8
user = False
home = None
root = None
prefix = None
distutils: /home/thu/.conda/envs/tf2/include/python3.8/UNKNOWN
sysconfig: /home/thu/.conda/envs/tf2/include/python3.8
user = False
home = None
root = None
prefix = None
You should consider upgrading via the '/home/thu/.conda/envs/tf2/bin/python -m pip install --upgrade pip' command.


/home/thu/.conda/envs/tf2/lib/python3.8/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


### MLE estimation function 

In [ ]:
def diag_term(X,i):
  arr0 = X[:,i].flatten()
  arr = arr0[~np.isnan(arr0)]
  return np.var(arr)

def musMLE(X,y,G):
    n,p = X.shape[0], X.shape[1]
    f = lambda g: np.nanmean(X[y==g,:],axis=0)
    musMLE = np.array([f(g) for g in range(G)])    
    return musMLE.T

def Smle(X,y,musMLE,g):
    '''
    function to compute the covariance matrix for the g-th class
    X: input, should be a numpy array
    y: label
    G: number of classes
    g: class index
    output:
    - mus: each row is a class mean
    - S: common covariance matrix of class 1,2,..., G 
    '''
    epsilon = 1e-5 # define epsilon to put r down to 0 if r < epsilon
    Xg, yg = X[y==g,:], y[y==g]
    n,p = Xg.shape[0], Xg.shape[1] 
 
    S = np.diag([diag_term(Xg,i) for i in range(p)]) 

    for i in range(p):      
      for j in range(i):
        if ((S[i,i] == 0.) | (S[j,j] == 0.)):
          S[i,j] = S[j,i] = 0.
          continue

        mat = Xg[:,[i,j]]

        # drop rows with NA
        idx = ~np.isnan(mat).any(axis=1)
        mat, y_arr = mat[idx], yg[idx]
        A = mg = len(y_arr) 

        s11 = mg*np.var(mat[:,0])
        s22 = mg*np.var(mat[:,1])
        s12 = sum((mat[:,0]-musMLE[i,g])*(mat[:,1]-musMLE[j,g]))
        B = S[i,i]*S[j,j]*A - s22 * S[i,i] - s11 * S[j,j]
        coefficient = [-A, s12, B, s12*S[i,i]*S[j,j]]
        r = np.roots(coefficient)
        r = r[abs(np.imag(r)) < epsilon]
        r = np.real(r)
        r[abs(r) < epsilon] = 0
 
        if len(r)>1:
          condi_var = S[j,j] - r**2/S[i,i]
          eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var
          r = r[eta == max(eta[~np.isnan(eta)])]

        if len(r) > 1: 
          if sum(r==0.0) == len(r):
            r = 0.
          else:  
            w = np.cov(mat, rowvar=False)  
            #r = r[w[0,1]*r>=0]
            r = r[np.abs(r-w[0,1]).argmin()] # select r that is closet to w[0,1] 

        S[i,j] = S[j,i] = r
    return S

## Individual compute error function 

In [ ]:
def err(mus, S, mus_est, S_est):
  er = [np.linalg.norm(mus_est-mus)/mus.size,
        np.linalg.norm(S_est.flatten().flatten()-S.flatten())/S.size]  
  return np.mean(er)  

def generate_nan(Xtrain, missing_rate):
  Xshape = Xtrain.shape
  na_id = np.random.randint(0,Xtrain.size,round(missing_rate*Xtrain.size))
  Xtr_nan = Xtrain.flatten()
  Xtr_nan[na_id] = np.nan 
  return Xtr_nan.reshape(Xshape) 

In [ ]:
def compute_err_dper(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]) 

    start = time.time()
    mus_mle = musMLE(Xtr_nan,y,G)
    S_mle = np.array([Smle(Xtr_nan,y,mus_mle, g) for g in range(G)])   
    mle_err = err(mus, S, mus_mle.T, S_mle)
    mle_time = time.time()-start
    
    return mle_err, mle_time

In [ ]:
def compute_err_soft(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]) 

    start = time.time()
    Xtr_softimpute = SoftImpute(max_iters = 100).fit_transform(Xtr_nan)
    mus_softimpute = np.asarray([np.mean(Xtr_softimpute[ytrain==g,:], axis=0
                                         ) for g in np.arange(G)])
    S_softimpute = np.asarray([np.cov(Xtr_softimpute[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])

    softimpute_err =  err(mus, S, mus_softimpute, S_softimpute)
    softimpute_time = time.time()-start
 
    return softimpute_err, softimpute_time

In [ ]:
def compute_err_mice(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]) 

    start = time.time()
    Xtr_mice = IterativeImputer(max_iter=100).fit(Xtr_nan).transform(Xtr_nan)
    mus_mice = np.asarray([np.mean(Xtr_mice[ytrain==g,:], axis=0
                                   ) for g in np.arange(G)])
    S_mice = np.asarray([np.cov(Xtr_mice[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])

    mice_err = err(mus, S, mus_mice, S_mice)
    mice_time = time.time()-start
 
    return mice_err, mice_time

In [ ]:
def compute_err_nuclear(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]) 

    start = time.time()
    Xtr_nuclear = NuclearNormMinimization(max_iters=100).fit_transform(Xtr_nan)
    mus_nuclear = np.asarray([np.mean(Xtr_nuclear[ytrain==g,:], axis=0
                                      ) for g in np.arange(G)])
    S_nuclear = np.asarray([np.cov(Xtr_nuclear[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    nuclear_err = err(mus, S, mus_nuclear, S_nuclear)
    nuclear_time = time.time()-start
 
    return nuclear_err, nuclear_time

In [ ]:
def compute_err_mforest(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]) 

    start = time.time()
    Xtr_mforest = MissForest(random_state=0).fit_transform(Xtr_nan)
    mus_mforest = np.asarray([np.mean(Xtr_mforest[ytrain==g,:], axis=0
                                      ) for g in np.arange(G)])
    S_mforest = np.asarray([np.cov(Xtr_mforest[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    mforest_err = err(mus, S, mus_mforest, S_mforest)
    mforest_time = time.time()-start      
 
    return mforest_err, mforest_time

### FASHION MNIST

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

fashion_mnist = tf.keras.datasets.fashion_mnist
(Xtrain, ytrain), (Xtest, ytest) = fashion_mnist.load_data()
Xtrain = Xtrain.astype(float).reshape((60000,784))
Xtest = Xtest.astype(float).reshape((10000,784))

X = np.vstack((Xtrain, Xtest))
y = np.hstack((ytrain, ytest))

# set random seed and shuffle the data
np.random.seed(1)
idx = np.arange(len(y))
np.random.shuffle(idx)
X, y = X[idx,:], y[idx]  

X.shape, y.shape 

((70000, 784), (70000,))

In [ ]:
G = 10
np.random.seed(3)
e20_dper = compute_err_dper(X, y, G, 0.2)
e35_dper = compute_err_dper(X, y, G, 0.35)
e50_dper = compute_err_dper(X, y, G, 0.5)
e65_dper = compute_err_dper(X, y, G, 0.65)
e80_dper = compute_err_dper(X, y, G, 0.8)

<ipython-input-2-cba8272b93b5>:54: RuntimeWarning: invalid value encountered in log
  eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var
<ipython-input-2-cba8272b93b5>:54: RuntimeWarning: invalid value encountered in log
  eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var
<ipython-input-2-cba8272b93b5>:54: RuntimeWarning: invalid value encountered in log
  eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var
<ipython-input-2-cba8272b93b5>:54: RuntimeWarning: invalid value encountered in log
  eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var
<ipython-input-2-cba8272b93b5>:54: RuntimeWarning: invalid value encountered in log
  eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var


In [ ]:
# the left column is the error, the right column is the running time
np.vstack((e20_dper,e35_dper,e50_dper,e65_dper,e80_dper))

array([[2.89677844e-05, 6.16749044e+03],
       [3.95722202e-05, 5.05816724e+03],
       [5.04169202e-05, 4.26305263e+03],
       [5.89859441e-05, 3.49414068e+03],
       [8.19900695e-05, 2.91714304e+03]])

## SOFT IMPUTE

In [ ]:
e20_soft = compute_err_soft(X, y, G, 0.2)
e35_soft = compute_err_soft(X, y, G, 0.35)
e50_soft = compute_err_soft(X, y, G, 0.5)
e65_soft = compute_err_soft(X, y, G, 0.65)
e80_soft = compute_err_soft(X, y, G, 0.8)

[SoftImpute] Max Singular Value of X_init = 2852.173305
[SoftImpute] Iter 1: observed MAE=0.127547 rank=784
[SoftImpute] Iter 2: observed MAE=0.130737 rank=784
[SoftImpute] Iter 3: observed MAE=0.130801 rank=690
[SoftImpute] Iter 4: observed MAE=0.125551 rank=590
[SoftImpute] Iter 5: observed MAE=0.122522 rank=554
[SoftImpute] Iter 6: observed MAE=0.121215 rank=542
[SoftImpute] Iter 7: observed MAE=0.120654 rank=535
[SoftImpute] Iter 8: observed MAE=0.120402 rank=532
[SoftImpute] Iter 9: observed MAE=0.120284 rank=530
[SoftImpute] Iter 10: observed MAE=0.120227 rank=528
[SoftImpute] Iter 11: observed MAE=0.120198 rank=528
[SoftImpute] Iter 12: observed MAE=0.120184 rank=528
[SoftImpute] Iter 13: observed MAE=0.120176 rank=528
[SoftImpute] Iter 14: observed MAE=0.120173 rank=528
[SoftImpute] Iter 15: observed MAE=0.120171 rank=528
[SoftImpute] Iter 16: observed MAE=0.120170 rank=528
[SoftImpute] Stopped after iteration 16 for lambda=57.043466
[SoftImpute] Max Singular Value of X_init = 

In [ ]:
# the left column is the error, the right column is the running time
np.vstack((e20_soft, e35_soft,e50_soft,e65_soft,e80_soft))

array([[3.77228654e-05, 7.08477541e+02],
       [6.59304443e-05, 9.27241102e+02],
       [9.49328728e-05, 1.12007290e+03],
       [1.27231684e-04, 1.66373290e+03],
       [1.62231823e-04, 2.01975027e+03]])

## MICE IMPUTE

In [ ]:
# start time: 7:42 AM, interrupt: 9:52pm
e20_mice = compute_err_mice(X, y, G, 0.2)

In [ ]:
# start time: 8:22 AM, interrupt: 1:30 PM
e35_mice = compute_err_mice(X, y, G, 0.35)

In [ ]:
# start time: 8:22 AM, interrupt: 1:30 PM
e50_mice = compute_err_mice(X, y, G, 0.5)

In [ ]:
# start time: 1:30 PM, interrupt: 7:00 PM
e65_mice = compute_err_mice(X, y, G, 0.65)

In [ ]:
# start time: 9:52 PM, interrupt: 8:22 AM
e80_mice = compute_err_mice(X, y, G, 0.8)

## Nuclear norm

In [ ]:
#kernel dies
e20_nuclear = compute_err_nuclear(X, y, G, 0.2)

In [ ]:
#kernel dies
e35_nuclear = compute_err_nuclear(X, y, G, 0.35)

In [ ]:
#kernel dies
e50_nuclear = compute_err_nuclear(X, y, G, 0.5)

In [ ]:
#kernel dies
e65_nuclear = compute_err_nuclear(X, y, G, 0.65)

In [ ]:
#kernel dies
e80_nuclear = compute_err_nuclear(X, y, G, 0.8)

## Miss forest

In [ ]:
G = 10
e20_mforest = compute_err_mforest(X, y, G, 0.2)

In [ ]:
compute_err_mforest(X, y, G, 0.35)

In [ ]:
compute_err_mforest(X, y, G, 0.5)

In [ ]:
compute_err_mforest(X, y, G, 0.65)

In [ ]:
compute_err_mforest(X, y, G, 0.8)